# Chatbot pentru răspunderea la întrebări frecvente



## Instalarea bibliotecilor necesare

In [1]:
# daca nu sunt deja instalate
# altfel se comenteaza
# %pip install numpy
# %pip install scikit-learn
# %pip install scipy
# %pip install nltk
# %pip install pandas
# %pip install matplotlib
# %pip install spacy
# %pip install sentence-transformers

## Prelucrarea datelor

1. Setul de date

Setul de date prevede mai multe coloane pentru a oferi o prespectivă mai largă pentru înțelegerea acestuia.

În cazul de față de interese sunt coloanele „Question” și „Answer”.

In [2]:
import pandas as pd

df = pd.read_csv("all_questions.txt", sep="\t")
df.head()

print(df.columns)

data = df[['Question', 'Answer']].dropna()

print(data)

Index(['ArticleTitle', 'Question', 'Answer', 'DifficultyFromQuestioner',
       'DifficultyFromAnswerer', 'ArticleFile'],
      dtype='object')
                                               Question  \
0     Was Abraham Lincoln the sixteenth President of...   
1     Was Abraham Lincoln the sixteenth President of...   
2     Did Lincoln sign the National Banking Act of 1...   
3     Did Lincoln sign the National Banking Act of 1...   
4                      Did his mother die of pneumonia?   
...                                                 ...   
3992          What areas do the Grevy's Zebras inhabit?   
3994  Which species of zebra is known as the common ...   
3995  Which species of zebra is known as the common ...   
3996                     At what age can a zebra breed?   
3997                     At what age can a zebra breed?   

                                                 Answer  
0                                                   yes  
1                              

2. Prelucrarea textului

Constă în:
* eliminarea duplicatelor,
* transformarea din uppercase în lowercase,
* eliminarea caracterelor care nu sunt cuvinte și care nu sunt whitespace-uri
* eliminarea cifrelor din text.

In [6]:
# eliminare duplicate

data["Question"] = data["Question"].drop_duplicates(keep="first")

print(f"Train:{len(data)}")

# transformare din uppercase in lowercase

data["Question"] = data["Question"].map(lambda x: x.lower() if isinstance(x, str) else x)

# eliminare a caracterelor care nu sunt cuvinte si care nu sunt whitespace-uri

data["Question"] = data["Question"].replace(to_replace=r'[^\w\s]', value='', regex=True)

# eliminare cifre din text 

data["Question"] = data["Question"].replace(to_replace=r'\d', value='', regex=True)

data = data.dropna(subset=["Question"])

print(f"Text curatat:")
print(data["Question"])

Train:2203
Text curatat:
0       [abraham, lincoln, sixteenth, president, unite...
2                 [lincoln, sign, national, banking, act]
4                                [mother, die, pneumonia]
6                [many, long, lincoln, formal, education]
8                     [lincoln, begin, political, career]
                              ...                        
3988                                         [zebra, eat]
3990                                        [zebra, hunt]
3992                       [area, grevys, zebra, inhabit]
3994                 [specie, zebra, know, common, zebra]
3996                                  [age, zebra, breed]
Name: Question, Length: 2148, dtype: object


3. Curatarea textului - Lemmatizarea textului

Lemmatizarea presupune eliminarea eventoalelor constructii sufixate sau prefixate pentru a aduce cuvântul la forma lui de bază.

Scopul utilizării este:

- uniformizare, cuvintele cu acceași rădăcină sunt tratate ca echivalente;
- reducerea dimensionalității;
- îmbunătățirea căutării/ comparării.

In [4]:
# tokenizare

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer(r'\w+')

data["Question"] = data["Question"].apply(lambda x: tokenizer.tokenize(x) if isinstance(x, str) else x)

# stergere stopwords

# poate necesita decomentare
# nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

data["Question"] = data["Question"].apply(lambda x: [word for word in x if word not in stop_words])

print("Text Prelucrat")
print(data["Question"])

Text Prelucrat
0       [abraham, lincoln, sixteenth, president, unite...
2                 [lincoln, sign, national, banking, act]
4                                [mother, die, pneumonia]
6               [many, long, lincolns, formal, education]
8                     [lincoln, begin, political, career]
                              ...                        
3988                                        [zebras, eat]
3990                                     [zebras, hunted]
3992                     [areas, grevys, zebras, inhabit]
3994               [species, zebra, known, common, zebra]
3996                                  [age, zebra, breed]
Name: Question, Length: 2203, dtype: object


In [5]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# pot necesita sa fie decomentate
# nltk.download('averaged_perceptron_tagger_eng')
# nltk.download('wordnet')

# lemmatizare
# initializare lemmatizer
lemmatizer = WordNetLemmatizer()

# functie pentru lemmatizare token-uri
def lemmatize_tokens(tokens):
    # conversie din pos in wordnet
    def get_wordnet_pos(word):
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)
    
    # lematizare token-uri
    lemmas = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    
    # returnare ca o lista
    return lemmas

data["Question"] = data["Question"].apply(lemmatize_tokens)

print("Text dupa Lemmatizare:")
print(data["Question"])

Text dupa Lemmatizare:
0       [abraham, lincoln, sixteenth, president, unite...
2                 [lincoln, sign, national, banking, act]
4                                [mother, die, pneumonia]
6                [many, long, lincoln, formal, education]
8                     [lincoln, begin, political, career]
                              ...                        
3988                                         [zebra, eat]
3990                                        [zebra, hunt]
3992                       [area, grevys, zebra, inhabit]
3994                 [specie, zebra, know, common, zebra]
3996                                  [age, zebra, breed]
Name: Question, Length: 2203, dtype: object


4. Word Embedding

Word Embeddings sunt reprezentări numerice ale cuvintelor într-un spațiu dimensional mai mic pentru a capta informația semantică și sintactică.

Word Embedding este o abordare pentru reprezentarea cuvintelor și a documentelor.
Word Embedding sau Word Vector este un vector numeric pentru a reprezenta un cuvânt într-un spațiu dimensional mai mic.

În cazul de față am ales o abordare bazată pe Contextualized Word Embeddings, mai precis BERT.

BERT este un model transformer care învață încorporări contextuale pentru cuvinte. Consideră întregul context al unui cuvânt ceea ce oferă o informație contextuală mai bogată decât dacă se folosesc frequency based embeddings, TF-IDF, sau prediction based word embedding, Word2Vec.

In [14]:
from sentence_transformers import SentenceTransformer

# reconstruire intrebari ca string
data["QuestionText"] = data["Question"].apply(lambda tokens: " ".join(tokens))

model = SentenceTransformer('all-MiniLM-L6-v2')

questions = data["QuestionText"].tolist()
answers = data["Answer"].tolist()
question_embeddings = model.encode(questions, convert_to_numpy=True)

print(question_embeddings)


[[-0.00719016  0.0577301  -0.01099544 ... -0.01460441 -0.05865231
   0.02819466]
 [-0.00987572 -0.01382468 -0.09109937 ... -0.02423994 -0.04201196
  -0.01090215]
 [ 0.00048814 -0.00686363  0.0077758  ... -0.05062495  0.09680961
   0.00913441]
 ...
 [-0.04268538  0.03998202 -0.02327148 ...  0.04347909  0.05327745
   0.00329734]
 [-0.09858818  0.02357402 -0.02428912 ...  0.04131453  0.07671772
   0.01495439]
 [-0.10551791  0.09273266 -0.00291227 ...  0.0076475   0.1136751
  -0.00361189]]


5. Calcularea gradului similaritatii

Se testează mai multe funcții pentru similaritate mai bună pentru întrebări

- cosine_similarity

$$
\mathrm{sim(X, Y)} = \frac{\sum_{i = 1}^{k} X_i * Y_i}{\sqrt{\sum_{i = 1}^{k} X_i^2} \sqrt{\sum_{i = 1}^{k} Y_i^2}}
$$

Cosine_similarity măsoară cosinusul unghiului dintre doi vectori în spațiu. În cazul prelucării de texx, este o metrică care nu este afectată de frecvența cuvintelor care apar în documente și este eficientă pentru a compara documente de diferite dimensiuni.

Este o metrică care nu este afectată de lungimea textului. Textele asimetrice pot avea un unghi mai mic între ele. Cu cât unghiul este mai mic cu atât similaritatea este mai mare. De asemenea, cosinusul are domeniul cuprins între `[-1, 1]`, iar din perspectiva NLP-urilor, valorile sunt cuprinse între `[0, 1]`. Dacă un cuvânt nu apare, atunci fracția devine 0.

- distanta euclidiana

$$
\mathrm{sim(X, Y) = \frac{1}{1 + \sqrt{\sum_{i = 1}^{k} (X_i - Y_i)^2}}}
$$

Distanța euclidiană reprezintă lungimea unui segment de dreaptă dintre două puncte. În cazul NLP-urilor, aceste puncte reprezintă cuvinte.

Cu cât distanța dintre două texte este mai mică, cu atât mai similare sunt acestea. Principala problemă cu această metrică este că lungimea textului afectează rezultatul. Propozițiile mai lungi tind să aibă un scor mai mare comparativ cu cele mai scurte.

- produsul scalar

$$
\mathrm{sim(X, Y) = \sum_{i = 1}^{k} X_i * Y_i}
$$

Produsul scalar este o măsură de proiecție și intuitiv arată cam cât de mult un vector este orientat în direcția celuilalt.

Va avea valori mari dacă vectorii au direcții similare și au o lungime mai mare.

O problemă care apare, la fel ca în cazul distanței euclidiene, este că depinde de lungimea vectorilor. De aceea produsul scalar nu este recomandat decât dacă s-a aplicat normalizarea pe respectivii vectori, caz în care, produsul scalar este totuna cu cosine_similarity.

In [19]:
import numpy as np
from numpy.linalg import norm

# inversul funcțiilor pentru a crește cu similaritatea

def cosine_sim(a, b):
    return (prod_scal(a, b) / (norm(a) * norm(b)))

def euclidean_distance(a, b):
    return 1 / (1 + np.sqrt(prod_scal(a - b, a - b)))

def prod_scal(A, B):
    return (sum(a * b for a, b in zip(A, B)))

6. Funcție best_match pentru o întrebare scrisă de utilizator și dicționarul de întrebări

Trebuiesc făcute exact aceleași operații de prelucrare și curățare de text așa cum au fost făcute pentru întrebările din setul de date.

In [20]:
import re

def best_match(u_question, method="cosine"):

    sim_func = {
        "cosine": cosine_sim,
        "euclidean": euclidean_distance,
        "prodscal": prod_scal
    }.get(method)

    text = u_question.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)
    tokens = tokenizer.tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    tokens = lemmatize_tokens(tokens)
    clean_text = " ".join(tokens)

    u_vec = model.encode(clean_text)

    scores = [sim_func(u_vec, q_vec) for q_vec in question_embeddings]
    idx = int(np.argmax(scores))

    return {
        "Input": u_question,
        "Matched Question": questions[idx],
        "Answer": answers[idx],
        "Score": float(scores[idx]),
        "Method": method
    }

In [21]:
# How old was Lincoln in 1816? seven

print(best_match("How old is Abraham Lincoln?", method="cosine"))
print(best_match("How old is Abraham Lincoln?", method="euclidean"))
print(best_match("How old is Abraham Lincoln?", method="prodscal"))

# How has Canada helped UN peacekeeping efforts? 

print(best_match("Did Canada help UN peacekeeping?", method="cosine"))
print(best_match("Did Canada help UN peacekeeping?", method="euclidean"))
print(best_match("Did Canada help UN peacekeeping?", method="prodscal"))

{'Input': 'How old is Abraham Lincoln?', 'Matched Question': 'old lincoln', 'Answer': 'seven', 'Score': 0.9362112283706665, 'Method': 'cosine'}
{'Input': 'How old is Abraham Lincoln?', 'Matched Question': 'old lincoln', 'Answer': 'seven', 'Score': 0.7368225455284119, 'Method': 'euclidean'}
{'Input': 'How old is Abraham Lincoln?', 'Matched Question': 'old lincoln', 'Answer': 'seven', 'Score': 0.9362112283706665, 'Method': 'prodscal'}
{'Input': 'Did Canada help UN peacekeeping?', 'Matched Question': 'canada help un peacekeeping effort', 'Answer': 'During the Suez Crisis of 1956, Lester B. Pearson eased tensions by proposing the inception of the United Nations Peacekeeping Force. Canada has since served in 50 peacekeeping missions, including every UN peacekeeping effort until 1989', 'Score': 0.9868934154510498, 'Method': 'cosine'}
{'Input': 'Did Canada help UN peacekeeping?', 'Matched Question': 'canada help un peacekeeping effort', 'Answer': 'During the Suez Crisis of 1956, Lester B. Pea

7. Input de la User

In [ ]:
while True:
    user_input = input("Intrebare:")
    
    if user_input.lower() in ["exit"]:
        break

    answer = best_match(user_input)
    print(answer["Answer"])